In [ ]:
# inicia bibliotecas e local do arquivo
import pandas as pd
import numpy as np
import re
df = pd.read_csv('./vendas_modificado.csv')
df.head(20)

In [1]:
#verifica os tipos
df.dtypes

NameError: name 'df' is not defined

In [ ]:
# verifica nulos
df.isnull().sum()

In [ ]:
# verifica os erros de digitação na coluna valor
erros_valor = df[~df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').str.strip().str.match(r'^\d+(\.\d+)?$')]


print(erros_valor[['valor']].head(10))

In [ ]:
# corige os erros dos valores
df = df[df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').str.strip().str.match(r'^\d+(\.\d+)?$')]

df['valor'] = df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').astype(float)



In [ ]:
#padroniza data hora e frete
df['data'] = pd.to_datetime(df['data'], errors='coerce')


df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

df['frete'] = df['frete'].fillna(0.0)

In [ ]:
# verifica nulos
df.isnull().sum()

In [ ]:
# dropa os nulos restantes 
df = df.dropna()


In [ ]:
# Corrige os valores nulos na coluna 'total' usando a fórmula correta
faltando_total = df['total'].isna()
df.loc[faltando_total, 'total'] = (df.loc[faltando_total, 'valor'] * df.loc[faltando_total, 'quantidade'] + df.loc[faltando_total, 'frete']).round(2)

# Recalcula todos os valores de 'total' e padroniza com duas casas decimais
df['total'] = (df['valor'] * df['quantidade'] + df['frete']).round(2)

# Remove linhas com valores negativos nas colunas especificadas
colunas = ['valor', 'quantidade', 'frete', 'total']
df = df[(df[colunas] >= 0).all(axis=1)].reset_index(drop=True)

# Exibe registros com discrepâncias
print(df[['valor', 'quantidade', 'frete', 'total', ]])




In [ ]:
# remove resultados maiores que 999 objetivo é de manter todos os resultaods no maximoa té a casa dos cem
df = df[df['total'] <= 999]

In [ ]:
# exibe o data frame
df.head(30)

In [ ]:
# Verificando os status 
df['status'].value_counts()

In [ ]:
#confere se ainda existem nulos
df.isnull().sum()

In [ ]:
# Corrigindo e padronizando os nomes dos status
df['status'] = df['status'].replace({
    'Pgto Confirmado': 'Pagamento Confirmado',
    'aguardando pagamento': 'Aguardando Pagamento',
    'Aguardando Pgto': 'Aguardando Pagamento',
    'AP': 'Aguardando Pagamento',
    'Sep': 'Em Separação',
    'Separando': 'Em Separação',
    'PC': 'Pagamento Confirmado',
    'Entg': 'Entregue',
    'Entregue com Sucesso': 'Entregue',
    'Transp': 'Em Transporte',
    'Transportando': 'Em Transporte'
})


In [ ]:
# Verificando como ficou a distribuição dos status após a padronização
df['status'].value_counts()


In [ ]:
df['pagamento'].value_counts()

In [ ]:
# verificando produtos
print(df['produto'].value_counts().to_string())


In [ ]:
# Dicionário de correções: chaves = nomes errados / valores = nomes corretos
correcoes_produtos = {
    # Corrigindo com símbolos e variações comuns
    'Pasta de Dente#$@!': 'Pasta de Dente',
    'Queijo Mussarela#$@!': 'Queijo Mussarela',
    'Sabonete#$@!': 'Sabonete',
    'Manteiga#$@!': 'Manteiga',
    'Café#$@!': 'Café',
    'Açúcar#$@!': 'Açúcar',
    'Desinfetante#$@!': 'Desinfetante',
    'Papel Toalha#$@!': 'Papel Toalha',
    'Condicionador#$@!': 'Condicionador',
    'Molho de Tomate#$@!': 'Molho de Tomate',
    'Água Mineral#$@!': 'Água Mineral',
    'Refrigerante#$@!': 'Refrigerante',
    'Presunto#$@!': 'Presunto',
    'Cerveja#$@!': 'Cerveja',
    'Biscoito Recheado#$@!': 'Biscoito Recheado',
    'Macarrão#$@!': 'Macarrão',
    'Óleo de Soja#$@!': 'Óleo de Soja',
    'Vinho#$@!': 'Vinho',
    'Sal#$@!': 'Sal',
    'Suco de Laranja#$@!': 'Suco de Laranja',
    'Shampoo#$@!': 'Shampoo',
    'Farinha de Trigo#$@!': 'Farinha de Trigo',
    'Detergente#$@!': 'Detergente',
    'Pão de Forma#$@!': 'Pão de Forma',
    'Arroz#$@!': 'Arroz',
    'Feijão#$@!': 'Feijão',
    'Amaciante#$@!': 'Amaciante',
    'Leite Integral#$@!': 'Leite Integral',
    'Papel Higiênico#$@!': 'Papel Higiênico',
    'Sabão em Pó#$@!': 'Sabão em Pó',
    'Carvão#$@!': 'Carvão',
    'Condicioiador#$@!': 'Condicionador',

    # Corrigindo erros de digitação
    'Refrigkrante': 'Refrigerante',
    'Scl': 'Sal',
    'Papel qoalha': 'Papel Toalha',
    'Desinfetanue': 'Desinfetante',
    'Qbeijo Mussarela': 'Queijo Mussarela',
    'Água Mineras': 'Água Mineral',
    'Presuntd': 'Presunto',
    'Macawrão': 'Macarrão',
    'Sucoyde Laranja': 'Suco de Laranja',
    'Clfé': 'Café',
    'Desinfekante': 'Desinfetante',
    'Majarrão': 'Macarrão',
    'Cnfé': 'Café',
    'Farinha de Tripo': 'Farinha de Trigo',
    'Manteigt': 'Manteiga',
    'zabonete': 'Sabonete',
    'Mqcarrão': 'Macarrão',
    'Arroc': 'Arroz',
    'tal': 'Sal',
    'Sabonepe': 'Sabonete',
    'Papel Twalha': 'Papel Toalha',
    'Água Mineual': 'Água Mineral',
    'Condibionador': 'Condicionador',
    'Caft': 'Café',
    'ieijão': 'Feijão',
    'Açúcaz': 'Açúcar',
    'Amaciayte': 'Amaciante',
    'Caff': 'Café',
    'Mopho de Tomate': 'Molho de Tomate',
    'Molmo de Tomate': 'Molho de Tomate',
    'Biscoitq Recheado': 'Biscoito Recheado',
    'Macirrão': 'Macarrão',
    'Deqergente': 'Detergente',
    'Cafc': 'Café',
    'Queijo Mussarelz': 'Queijo Mussarela',
    'Deterwente': 'Detergente',
}

# Aplicando as correções no DataFrame
df['produto'] = df['produto'].replace(correcoes_produtos)


In [ ]:
# verificando cidades
df['cidade'].value_counts()

In [ ]:
# verificando pais
df['pais'].value_counts()

In [ ]:
# verificando marcas
print(df['marca'].value_counts().to_string())

In [ ]:
# verificando clientes
print(df['cliente'].value_counts().to_string())

In [ ]:
# remove espaços das colunas , espaços extras dos valores
df.columns = df.columns.str.strip()
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# exibe o data frame
df.head(50)


In [ ]:
# dropa duplicatas
df = df.drop_duplicates()

In [ ]:

# Define uma função para validar o CEP
def cep_valido(cep):
    if isinstance(cep, str):
        return bool(re.fullmatch(r"\d{5}-\d{3}", cep))
    return False

# Aplica o filtro
df = df[df['cep'].apply(cep_valido)].reset_index(drop=True)

# Formata os CEPs para o padrão 00000-000
df['cep'] = df['cep'].astype(str).str.zfill(8).str.replace(r'(\d{5})(\d{3})', r'\1-\2', regex=True)


In [ ]:
# transforma o data frame em csv
df = df.dropna().reset_index(drop=True)

df.to_csv('vendas_tratado.csv', index=False)


# Relatório de Limpeza e Padronização de Dados

Durante o processo de análise e tratamento dos dados, diversos problemas que comprometiam a integridade e a qualidade das informações foram identificados e corrigidos. Essas ações visaram padronizar os formatos e assegurar que o conjunto de dados seja confiável para análises futuras. A seguir, apresenta-se um resumo detalhado das etapas realizadas:

---

## 1. Leitura e Inspeção Inicial

- **Importação:** O arquivo `vendas_modificado.csv` foi importado utilizando a biblioteca `pandas`.
- **Estrutura e Tipos de Dados:** Foram verificados os tipos de dados e a presença de valores nulos, o que permitiu identificar possíveis inconsistências iniciais.

---

## 2. Tratamento da Coluna `valor`

- **Identificação de Erros:** Foram encontrados problemas de digitação e formatação, como a presença do símbolo `R$` e o uso de vírgula para separar decimais.
- **Limpeza dos Dados:** As entradas que não correspondiam ao padrão esperado foram removidas.
- **Conversão de Tipo:** Após a limpeza, os valores foram convertidos com sucesso para o tipo `float`.

---

## 3. Padronização de Datas, Horários e Valores de Frete

- **Datas:** A coluna `data` foi convertida para o formato `datetime`, uniformizando a representação temporal.
- **Horários:** A coluna `hora` foi ajustada para o padrão `HH:MM:SS`.
- **Frete:** Os valores nulos na coluna `frete` foram substituídos por `0.0`, garantindo que os cálculos subsequentes não fossem comprometidos.

---

## 4. Correção da Coluna `total`

- **Recalculo dos Valores:** Utilizando a fórmula `total = valor * quantidade + frete`, os valores ausentes foram recalculados.
- **Padronização e Validação:** Todos os valores foram recalculados, arredondados para duas casas decimais, e linhas com valores negativos foram removidas para manter a coerência dos dados.

---

## 5. Eliminação de Duplicatas e Registros com Valores Faltantes

- **Remoção de Valores Nulos Restantes:** Registros que ainda continham valores nulos após as etapas anteriores foram eliminados.
- **Eliminação de Duplicatas:** Foram removidas entradas duplicadas para evitar redundâncias na base de dados.

---

## 6. Padronização dos Status de Pagamento

- **Correção de Inconsistências:** A coluna `status` passava por diversas variações (por exemplo, _Pgto Confirmado_, _AP_, _aguardando pagamento_) que foram unificadas.
- **Categorias Definidas:** Após a padronização, os status foram categorizados como:
  - **Pagamento Confirmado**
  - **Aguardando Pagamento**
  - **Em Separação**
  - **Em Transporte**
  - **Entregue**

---

## 7. Correção de Nomes de Produtos

- **Padronização dos Nomes:** Muitos produtos apresentavam caracteres especiais ou erros de digitação. Um dicionário de correções foi criado para mapear os nomes errados para suas formas corretas.
- **Resultado:** Essa etapa facilitou a análise das vendas, possibilitando uma identificação precisa dos produtos.

---

## 8. Análise das Informações Complementares

- **Verificações:** Foram analisadas outras colunas, como `cidade`, `país`, `marca` e `cliente`, para assegurar a consistência dos dados.
- **Ajustes de Formatação:** Foram removidos espaços extras nos nomes das colunas e nos valores, garantindo a padronização.

---

## 9. Validação e Formatação de CEP

- **Validação do CEP:** Uma função foi criada para validar se os CEPs obedecem o padrão `00000-000`.
- **Ajuste de Formato:** Os CEPs foram formatados e validados, assegurando a integridade dos dados de localização.

---

## Conclusão

**Resultado Final:**

- **Consistência dos Dados:** Todos os tipos e formatos foram padronizados.
- **Cálculos Precisos:** Os valores foram recalculados e validados, garantindo a precisão de campos como `total`.
- **Eliminação de Inconsistências:** Dados nulos, duplicados e inconsistências em nomes foram completamente eliminados ou corrigidos.


